# GMRES Algorithm: Generalized Minimal Residual Method

## Introduction

The **Generalized Minimal Residual (GMRES)** method is an iterative algorithm for solving large, sparse, non-symmetric linear systems of the form:

$$A\mathbf{x} = \mathbf{b}$$

where $A \in \mathbb{R}^{n \times n}$ is a non-singular matrix (not necessarily symmetric), $\mathbf{b} \in \mathbb{R}^n$ is the right-hand side vector, and $\mathbf{x} \in \mathbb{R}^n$ is the solution vector.

Developed by Saad and Schultz in 1986, GMRES is one of the most widely used Krylov subspace methods in computational science and engineering.

## Theoretical Foundation

### Krylov Subspace

GMRES constructs an approximate solution $\mathbf{x}_k$ from the **Krylov subspace**:

$$\mathcal{K}_k(A, \mathbf{r}_0) = \text{span}\{\mathbf{r}_0, A\mathbf{r}_0, A^2\mathbf{r}_0, \ldots, A^{k-1}\mathbf{r}_0\}$$

where $\mathbf{r}_0 = \mathbf{b} - A\mathbf{x}_0$ is the initial residual.

### Minimization Property

At each iteration $k$, GMRES finds $\mathbf{x}_k \in \mathbf{x}_0 + \mathcal{K}_k(A, \mathbf{r}_0)$ that minimizes the 2-norm of the residual:

$$\mathbf{x}_k = \arg\min_{\mathbf{x} \in \mathbf{x}_0 + \mathcal{K}_k} \|\mathbf{b} - A\mathbf{x}\|_2$$

This is equivalent to:

$$\mathbf{x}_k = \mathbf{x}_0 + \arg\min_{\mathbf{z} \in \mathcal{K}_k} \|\mathbf{r}_0 - A\mathbf{z}\|_2$$

### Arnoldi Process

GMRES uses the **Arnoldi iteration** to construct an orthonormal basis $\{\mathbf{v}_1, \mathbf{v}_2, \ldots, \mathbf{v}_k\}$ for the Krylov subspace. The process generates:

$$AV_k = V_{k+1}\bar{H}_k$$

where:
- $V_k = [\mathbf{v}_1, \mathbf{v}_2, \ldots, \mathbf{v}_k] \in \mathbb{R}^{n \times k}$ has orthonormal columns
- $\bar{H}_k \in \mathbb{R}^{(k+1) \times k}$ is an upper Hessenberg matrix

The entries of $\bar{H}_k$ are computed via:

$$h_{i,j} = \mathbf{v}_i^T A \mathbf{v}_j, \quad i = 1, \ldots, j+1$$

### Least Squares Problem

Writing $\mathbf{z} = V_k \mathbf{y}$ for $\mathbf{y} \in \mathbb{R}^k$, the minimization becomes:

$$\min_{\mathbf{y} \in \mathbb{R}^k} \|\beta \mathbf{e}_1 - \bar{H}_k \mathbf{y}\|_2$$

where $\beta = \|\mathbf{r}_0\|_2$ and $\mathbf{e}_1 = [1, 0, \ldots, 0]^T \in \mathbb{R}^{k+1}$.

This least squares problem is efficiently solved using **Givens rotations** to transform $\bar{H}_k$ into upper triangular form.

### Convergence Properties

GMRES is guaranteed to converge in at most $n$ iterations for a non-singular matrix $A$. The convergence rate depends on the spectrum of $A$:

$$\frac{\|\mathbf{r}_k\|_2}{\|\mathbf{r}_0\|_2} \leq \min_{p \in \mathcal{P}_k, p(0)=1} \max_{\lambda \in \sigma(A)} |p(\lambda)|$$

where $\mathcal{P}_k$ is the set of polynomials of degree at most $k$ and $\sigma(A)$ is the spectrum of $A$.

**Key factors affecting convergence:**
- Clustering of eigenvalues away from zero
- Condition number $\kappa(A) = \|A\|_2 \|A^{-1}\|_2$
- Field of values (numerical range) of $A$

## Implementation

We now implement GMRES from scratch, demonstrating the core algorithm components.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve_triangular
from scipy.sparse import diags, csr_matrix
from scipy.sparse.linalg import gmres as scipy_gmres

np.random.seed(42)

In [ ]:
def arnoldi_iteration(A, v, k):
    """
    Perform k steps of the Arnoldi iteration.
    
    Parameters:
    -----------
    A : ndarray or sparse matrix
        The system matrix (n x n)
    v : ndarray
        Initial vector (will be normalized)
    k : int
        Number of Arnoldi iterations
        
    Returns:
    --------
    V : ndarray
        Orthonormal basis vectors (n x (k+1))
    H : ndarray
        Upper Hessenberg matrix ((k+1) x k)
    """
    n = len(v)
    V = np.zeros((n, k + 1))
    H = np.zeros((k + 1, k))
    
    # Normalize initial vector
    V[:, 0] = v / np.linalg.norm(v)
    
    for j in range(k):
        # Matrix-vector product
        w = A @ V[:, j]
        
        # Modified Gram-Schmidt orthogonalization
        for i in range(j + 1):
            H[i, j] = np.dot(V[:, i], w)
            w = w - H[i, j] * V[:, i]
        
        H[j + 1, j] = np.linalg.norm(w)
        
        # Check for breakdown
        if H[j + 1, j] < 1e-12:
            return V[:, :j+2], H[:j+2, :j+1]
        
        V[:, j + 1] = w / H[j + 1, j]
    
    return V, H

In [ ]:
def apply_givens_rotation(h, cs, sn, k):
    """
    Apply previous Givens rotations and compute new rotation.
    
    Parameters:
    -----------
    h : ndarray
        Column of Hessenberg matrix
    cs : ndarray
        Cosine values of Givens rotations
    sn : ndarray
        Sine values of Givens rotations
    k : int
        Current iteration
        
    Returns:
    --------
    h : ndarray
        Transformed column
    cs : ndarray
        Updated cosines
    sn : ndarray
        Updated sines
    """
    # Apply previous rotations
    for i in range(k):
        temp = cs[i] * h[i] + sn[i] * h[i + 1]
        h[i + 1] = -sn[i] * h[i] + cs[i] * h[i + 1]
        h[i] = temp
    
    # Compute new rotation
    r = np.sqrt(h[k]**2 + h[k + 1]**2)
    if r < 1e-12:
        cs[k] = 1.0
        sn[k] = 0.0
    else:
        cs[k] = h[k] / r
        sn[k] = h[k + 1] / r
    
    # Apply new rotation
    h[k] = r
    h[k + 1] = 0.0
    
    return h, cs, sn

In [ ]:
def gmres(A, b, x0=None, tol=1e-10, max_iter=None):
    """
    GMRES algorithm for solving Ax = b.
    
    Parameters:
    -----------
    A : ndarray or sparse matrix
        System matrix (n x n)
    b : ndarray
        Right-hand side vector
    x0 : ndarray, optional
        Initial guess (default: zero vector)
    tol : float
        Convergence tolerance
    max_iter : int, optional
        Maximum iterations (default: n)
        
    Returns:
    --------
    x : ndarray
        Approximate solution
    residuals : list
        Residual norms at each iteration
    """
    n = len(b)
    if max_iter is None:
        max_iter = n
    if x0 is None:
        x0 = np.zeros(n)
    
    # Initial residual
    r0 = b - A @ x0
    beta = np.linalg.norm(r0)
    
    if beta < tol:
        return x0, [beta]
    
    # Initialize arrays
    V = np.zeros((n, max_iter + 1))
    H = np.zeros((max_iter + 1, max_iter))
    cs = np.zeros(max_iter)
    sn = np.zeros(max_iter)
    g = np.zeros(max_iter + 1)
    g[0] = beta
    
    V[:, 0] = r0 / beta
    residuals = [beta]
    
    for k in range(max_iter):
        # Arnoldi step
        w = A @ V[:, k]
        
        # Modified Gram-Schmidt
        for i in range(k + 1):
            H[i, k] = np.dot(V[:, i], w)
            w = w - H[i, k] * V[:, i]
        
        H[k + 1, k] = np.linalg.norm(w)
        
        # Apply Givens rotations
        h_col = H[:k + 2, k].copy()
        h_col, cs, sn = apply_givens_rotation(h_col, cs, sn, k)
        H[:k + 2, k] = h_col
        
        # Update residual norm
        g[k + 1] = -sn[k] * g[k]
        g[k] = cs[k] * g[k]
        
        residual = abs(g[k + 1])
        residuals.append(residual)
        
        # Check convergence
        if residual < tol:
            break
        
        # Breakdown check
        if H[k + 1, k] < 1e-12:
            break
        
        V[:, k + 1] = w / H[k + 1, k]
    
    # Solve upper triangular system
    k_final = len(residuals) - 1
    y = solve_triangular(H[:k_final, :k_final], g[:k_final], lower=False)
    
    # Compute solution
    x = x0 + V[:, :k_final] @ y
    
    return x, residuals

## Numerical Experiments

### Test Problem 1: Convection-Diffusion Equation

We create a sparse non-symmetric matrix arising from the 1D convection-diffusion equation:

$$-\epsilon u''(x) + c u'(x) = f(x)$$

discretized with central differences on a uniform grid.

In [ ]:
def create_convection_diffusion_matrix(n, epsilon=0.01, c=1.0):
    """
    Create 1D convection-diffusion matrix.
    
    Parameters:
    -----------
    n : int
        Matrix dimension
    epsilon : float
        Diffusion coefficient
    c : float
        Convection coefficient
        
    Returns:
    --------
    A : sparse matrix
        Convection-diffusion matrix
    """
    h = 1.0 / (n + 1)
    
    # Diffusion term: -epsilon * u''
    main_diag = 2 * epsilon / h**2 * np.ones(n)
    off_diag = -epsilon / h**2 * np.ones(n - 1)
    
    # Convection term: c * u' (central difference)
    upper_conv = c / (2 * h) * np.ones(n - 1)
    lower_conv = -c / (2 * h) * np.ones(n - 1)
    
    A = diags([off_diag + lower_conv, main_diag, off_diag + upper_conv], 
              [-1, 0, 1], format='csr')
    
    return A

In [ ]:
# Problem setup
n = 100
A_cd = create_convection_diffusion_matrix(n, epsilon=0.01, c=1.0)

# True solution and right-hand side
x_true = np.sin(np.pi * np.linspace(0, 1, n + 2)[1:-1])
b_cd = A_cd @ x_true

# Solve with our GMRES implementation
x_gmres, residuals_gmres = gmres(A_cd, b_cd, tol=1e-10, max_iter=100)

print(f"Problem size: {n}")
print(f"Number of iterations: {len(residuals_gmres) - 1}")
print(f"Final residual: {residuals_gmres[-1]:.2e}")
print(f"Solution error: {np.linalg.norm(x_gmres - x_true):.2e}")

### Test Problem 2: Effect of Convection Strength

We examine how the convection coefficient $c$ (which controls the non-symmetry) affects convergence.

In [ ]:
# Test different convection strengths
convection_values = [0.1, 0.5, 1.0, 2.0]
n = 50

plt.figure(figsize=(12, 5))

# Subplot 1: Convergence history
plt.subplot(1, 2, 1)
for c in convection_values:
    A = create_convection_diffusion_matrix(n, epsilon=0.01, c=c)
    x_true = np.ones(n)
    b = A @ x_true
    _, residuals = gmres(A, b, tol=1e-12, max_iter=n)
    
    # Normalize residuals
    residuals_norm = np.array(residuals) / residuals[0]
    plt.semilogy(residuals_norm, label=f'c = {c}', linewidth=2)

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Relative Residual $\\|r_k\\|/\\|r_0\\|$', fontsize=12)
plt.title('GMRES Convergence: Effect of Convection Strength', fontsize=14)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)

# Subplot 2: Eigenvalue distribution
plt.subplot(1, 2, 2)
colors = plt.cm.viridis(np.linspace(0, 0.8, len(convection_values)))
for c, color in zip(convection_values, colors):
    A = create_convection_diffusion_matrix(n, epsilon=0.01, c=c)
    eigenvalues = np.linalg.eigvals(A.toarray())
    plt.scatter(eigenvalues.real, eigenvalues.imag, 
                alpha=0.6, s=30, label=f'c = {c}', color=color)

plt.xlabel('Real Part', fontsize=12)
plt.ylabel('Imaginary Part', fontsize=12)
plt.title('Eigenvalue Distribution', fontsize=14)
plt.legend(loc='best')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("Plot saved to 'plot.png'")

### Analysis of Results

The plots reveal several key aspects of GMRES behavior:

1. **Convergence rate depends on spectrum**: As the convection strength $c$ increases, the eigenvalues spread further into the complex plane, generally slowing convergence.

2. **Non-symmetric effects**: The imaginary parts of eigenvalues (caused by the convection term) indicate the degree of non-symmetry. GMRES handles this naturally, unlike methods requiring symmetry.

3. **Eigenvalue clustering**: When eigenvalues cluster away from zero, GMRES converges faster because the minimum residual polynomial can more easily make all eigenvalues small.

### Comparison with SciPy Implementation

Let's verify our implementation against SciPy's optimized GMRES.

In [ ]:
# Larger problem
n = 200
A = create_convection_diffusion_matrix(n, epsilon=0.005, c=1.5)
x_true = np.sin(2 * np.pi * np.linspace(0, 1, n + 2)[1:-1])
b = A @ x_true

# Our implementation
x_ours, residuals_ours = gmres(A, b, tol=1e-10, max_iter=200)

# SciPy implementation
residuals_scipy = []
def callback(rk):
    residuals_scipy.append(rk)

x_scipy, info = scipy_gmres(A, b, rtol=1e-10, maxiter=200, callback=callback, callback_type='pr_norm')

print("Comparison of GMRES Implementations")
print("=" * 40)
print(f"Our implementation:")
print(f"  Iterations: {len(residuals_ours) - 1}")
print(f"  Final residual: {residuals_ours[-1]:.2e}")
print(f"  Solution error: {np.linalg.norm(x_ours - x_true):.2e}")
print(f"\nSciPy implementation:")
print(f"  Iterations: {len(residuals_scipy)}")
print(f"  Final residual: {residuals_scipy[-1] if residuals_scipy else 'N/A':.2e}")
print(f"  Solution error: {np.linalg.norm(x_scipy - x_true):.2e}")

## Key Takeaways

1. **GMRES minimizes the residual norm** over the Krylov subspace at each iteration, guaranteeing monotonic decrease.

2. **The Arnoldi process** builds an orthonormal basis efficiently, requiring only one matrix-vector product per iteration.

3. **Givens rotations** transform the least squares problem into an easily solvable triangular system.

4. **Memory considerations**: Full GMRES stores all basis vectors, requiring $O(nk)$ storage for $k$ iterations. Restarted GMRES variants address this limitation.

5. **Preconditioning** can dramatically accelerate convergence by clustering eigenvalues and reducing the condition number.

## References

1. Saad, Y., & Schultz, M. H. (1986). GMRES: A generalized minimal residual algorithm for solving nonsymmetric linear systems. *SIAM Journal on Scientific and Statistical Computing*, 7(3), 856-869.

2. Saad, Y. (2003). *Iterative Methods for Sparse Linear Systems* (2nd ed.). SIAM.

3. Trefethen, L. N., & Bau, D. (1997). *Numerical Linear Algebra*. SIAM.